<a href="https://colab.research.google.com/github/mtafadar/react-complete-guide-code/blob/master/CS483HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS483 - Homework 2
## Spark Tutorial

Complete you student information here:

You Name: Mosrour Tafadar

UIN: 657904373

Net ID: mtafad2

In this tutorial, you will learn how to use [Apache Spark](https://spark.apache.org) in local mode on a Colab enviroment. Credits to [Stanford CS246](https://web.stanford.edu/class/cs246/). 

The tutorial is worth 5 points. The bonus question is worth extra 3 points, which can be added to the total score of this course. After you complete this homework, submit pdf file to gradescope AND ipynb file to blackboard.

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 11.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=003952ce6fa5d428f1af9030e7061d2e95d83caa5fe005d988e3d02711d36e9a
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The followin

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Now, we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1L6pCQkldvdBoaEhRFzL0VnrggEFvqON4'
downloaded = drive.CreateFile({'id': id}) 
downloaded.GetContentFile('Bombing_Operations.json.gz')

id='14dyBmcTBA32uXPxDbqr0bFDIzGxMTWwl'
downloaded = drive.CreateFile({'id': id}) 
downloaded.GetContentFile('Aircraft_Glossary.json.gz')  

If you executed the cells above, you should be able to see the files *Bombing_Operations.json.gz* and *Aircraft_Glossary.json.gz* under the "Files" tab on the left panel.

In [5]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.


In [6]:
# create the session

conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()


You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [7]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to check the Spark UI.

**To facilitate setting up the ngrok tunnel, please do the following steps:**
1. First, navigate to the ngrok website and create your account: https://dashboard.ngrok.com/login
2. Then, obtain your authentication token from https://dashboard.ngrok.com/get-started/your-authtoken
3. Replace \<YOUR_AUTH_TOKEN_HERE\> in the following cell with your auth token.

You can then proceed to run the following cells, and you should be able to see the Spark UI as a separate web page.

(If you see a security warning, "Deceptive site ahead", please click "Details -> visit this unsafe site" to view the Spark UI.)

In [10]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!./ngrok authtoken 2KZdPX6tyosODYlevgLiiEjTRUE_6FyUqBo8q9Cds8aYaBVv8
!cat ~/.ngrok2/ngrok.yml
get_ipython().system_raw('./ngrok http 4050 &')

--2023-02-15 04:27:33--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.28M  12.9MB/s    in 1.0s    

2023-02-15 04:27:35 (12.9 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
authtoken: 2KZdPX6tyosODYlevgLiiEjTRUE_6FyUqBo8q9Cds8aYaBVv8


In [11]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://d17f-34-141-146-98.ngrok.io


Let's start from here 

In [13]:
def read_text_file(filename):
    with open(filename) as f:
        lines = f.readlines()
    data = {}
    for line in lines:
        parts = line.strip().split("\t")
        index = int(parts[0])
        values = list(map(int, parts[1].split(",")))
        data[index] = values
    return data

data = read_text_file("temp.txt")
data

{0: [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94],
 1: [0,
  5,
  20,
  135,
  2409,
  8715,
  8932,
  10623,
  12347,
  12846,
  13840,
  13845,
  14005,
  20075,
  21556,
  22939,
  23520,
  28193,
  29724,
  29791,
  29826,
  30691,
  31232,
  31435,
  32317,
  32489,
  34394,
  35589,
  35605,
  35606,
  35613,
  35633,
  35648,
  35678,
  38737,
  43447,
  44846,
  44887,
  49226,
  49985,
  623,
  629,
  4999,
  6156,
  13912,
  14248,
  15190,
  17636,
  19217,
  20074,
  27536,
 

In [20]:
# function to create a dictionary from input data
def create_dict(line):
    fields = line.split('\t')
    user = fields[0]
    friends = fields[1].split(',')
    return (user, friends)

# function to create a list of mutual friends
def create_mutual_friends(user, friends, user_dict):
    mutual_friends = []
    for friend in friends:
        friend_friends = set(user_dict.get(friend, []))
        mutual_friends.extend(list(friend_friends.intersection(set(friends))))
    return (user, list(set(mutual_friends)))

# function to sort recommendations by mutual friends count and user ID
def sort_recommendations(user, recommendations):
    return (user, sorted(recommendations, key=lambda x: (-x[1], x[0]))[:10])



In [22]:
lines = sc.textFile("temp2.txt")
user_dict = lines.map(create_dict).collectAsMap()

mutual_friends = lines.map(lambda x: create_mutual_friends(x.split('\t')[0], x.split('\t')[1].split(','), user_dict)).mapValues(lambda x: [friend for friend in x if friend not in user_dict[x[0]]])

recommendations = mutual_friends.map(lambda x: (x[0], [(friend, len(set(user_dict[friend]).intersection(set(x[1])))) for friend in x[1] if friend in user_dict])).\
map(lambda x: sort_recommendations(x[0], x[1])).sortByKey()

recommendations.map(lambda x: "{}\t{}".format(x[0], ','.join(str(friend[0]) for friend in x[1]))).saveAsTextFile("output.txt")

sc.stop()